In [ ]:
!pip install trl

In [ ]:
!git --version

git version 2.34.1


In [2]:
!git lfs install
!git clone https://huggingface.co/Qwen/Qwen3-0.6B

Git LFS initialized.
Cloning into 'Qwen3-0.6B'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 80 (delta 36), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (80/80), 1.75 MiB | 4.57 MiB/s, done.


In [ ]:
!nvidia-smi


Mon Aug 18 21:30:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [18]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer

In [29]:

model_path = "./Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)


In [20]:
# 2. LoRA ayarları
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16,
    target_modules=["q_proj","v_proj"], lora_dropout=0.05
)


In [21]:
model = get_peft_model(model, lora_config)



# Import dataset

In [22]:
# 3. Veri (örnek: CodeAlpaca)
dataset = load_dataset("flytech/python-codes-25k")



# Training configs

In [25]:
# Tokenize before dataset
def tokenize_function(example):
    return tokenizer(example["instruction"], truncation=True, max_length=512)

tokenized_dataset = dataset["train"].map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./Qwen3-0.6B-lora-python-expert-training",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    report_to=[]   # 👈 To close W&B

)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

Map:   0%|          | 0/49626 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/49626 [00:00<?, ? examples/s]

In [26]:
trainer.train()

Step,Training Loss
100,1.828800
200,1.785400
300,1.746900
400,1.705200
500,1.713300
600,1.690600
700,1.705500
800,1.658800
900,1.704600
1000,1.597600


TrainOutput(global_step=3102, training_loss=1.6443452346255747, metrics={'train_runtime': 4809.123, 'train_samples_per_second': 10.319, 'train_steps_per_second': 0.645, 'total_flos': 5307835129135104.0, 'train_loss': 1.6443452346255747})

# Merge LoRa with base model

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. Base model
base_model = AutoModelForCausalLM.from_pretrained(
    "./Qwen3-0.6B",
    torch_dtype="auto",
    device_map="auto"
)

# 2. Tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Qwen3-0.6B")

# 3. get LoRA
model = PeftModel.from_pretrained(base_model, "./Qwen3-0.6B-lora-python-expert-training/checkpoint-3102")

# 4. merge LoRA + Base
merged_model = model.merge_and_unload()

# 5. Save single file
save_path = "./Qwen3-0.6B-lora-python-expert"
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./Qwen3-0.6B-alpaca/tokenizer_config.json',
 './Qwen3-0.6B-alpaca/special_tokens_map.json',
 './Qwen3-0.6B-alpaca/chat_template.jinja',
 './Qwen3-0.6B-alpaca/vocab.json',
 './Qwen3-0.6B-alpaca/merges.txt',
 './Qwen3-0.6B-alpaca/added_tokens.json',
 './Qwen3-0.6B-alpaca/tokenizer.json')

In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path_fine_tuned = "./Qwen3-0.6B-lora-python-expert"  # fine-tune ettiysen output klasörü de olabilir

tokenizer_fine_tuned = AutoTokenizer.from_pretrained(model_path_fine_tuned)
model_fine_tuned = AutoModelForCausalLM.from_pretrained(model_path_fine_tuned, torch_dtype="auto", device_map="auto")


In [30]:
import torch

def chat(model, tokenizer, prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




# Fine-tuned model with python dataset

In [31]:
# Example on fine-tuned model
response = chat(model_fine_tuned, tokenizer, "Write a Python function, this function should return prime numbers between 0-100")
print(response)

Write a Python function, this function should return prime numbers between 0-1000. The function should be optimized for the time complexity. def primeNumbers(limit): 
    primeNumbers = []
    for i in range(0, limit+1):
        if isPrime(i):
            primeNumbers.append(i)
    return primeNumbers

def isPrime(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

# Call the function
primeNumbers(1000) # Output: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 


# Base model

In [32]:
# Example on normal model
response = chat(model, tokenizer, "Write a Python function called `prime_structural_member` that takes a list of integers as input.")
print(response)

Write a Python function called `prime_structural_member` that takes a list of integers as input. The function should return a list of integers that consists of all the prime numbers in the list and their respective positions. If no prime numbers exist, return an empty list.

For example, if the input is [5, 2, 4, 6, 7, 8, 9], the output should be [5, 2, 7, 3, 5, 1, 3].

Also, note that the input list could have elements with 0 or negative numbers. The function should not process those, and only consider the absolute values of the input elements.

First, you should read the input list from the user and then process it to return the result as specified.

So, what is the approach to solve this problem?

First, we can read the input list as a variable. Then, we can loop through each element in the list. For each element, check if it is a prime number. If it is, add it to a list


# Copy to Google Drive

In [34]:
!cp -r /content/Qwen3-0.6B-lora-python-expert-training /content/drive/MyDrive/python/Qwen3-0.6B-lora-python-expert
